In [33]:
# coding: utf-8

from  __future__ import absolute_import
from __future__ import print_function
from ImageDataGeneratorCustom import ImageDataGeneratorCustom
import numpy as np
from keras.applications.vgg16 import VGG16
from keras.layers import *
from keras.models import Model, load_model
from keras.preprocessing.image import load_img, img_to_array
import tensorflow as tf
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
import tensorflow_hub as hub
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingRegressor

import tensorflow as tf
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from keras import layers
from keras import optimizers
import shutil
import random
from PIL import Image
import numpy as np
import logging
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from keras import layers
import tensorflow_hub as hub
import shutil
import random
from PIL import Image
import pickle
import shutil
from tensorflow import debugging
import matplotlib.pyplot as plt
import numpy as np
import os
import random
import tensorflow as tf
from pathlib import Path
from keras import applications
from keras import layers
from keras import losses
from keras import optimizers
from keras import metrics
from keras import Model
from keras.applications import resnet
from keras.callbacks import ModelCheckpoint
import tensorflow_addons as tfa
from keras.regularizers import l2
from keras.activations import *


import logging
logger = tf.get_logger()
logger.setLevel(logging.ERROR)


print(tf.__version__)

# tf.config.run_functions_eagerly(True)
# tf.data.experimental.enable_debug_mode()

model_path = "./deep_ranking"

# batch_size = 96
# batch_size = 8
batch_size = 24
# batch_size = 3
# batch_size = 1

2.8.0


# Model

a lot of the code comes from https://keras.io/examples/vision/siamese_network/
and https://github.com/akarshzingade/image-similarity-deep-ranking/blob/master/deepRanking.py

In [34]:


embedding = tf.keras.Sequential([
    # tf.keras.layers.Dense(512, activation='relu'),
    # tf.keras.layers.BatchNormalization(),
    # tf.keras.layers.Dense(256, activation='relu'),
    # tf.keras.layers.BatchNormalization(),
    # tf.keras.layers.Dense(256, activation='linear'),

    # GlobalAveragePooling2D(),

    # tf.keras.layers.Dense(1024, activation='linear'),


    Dense(4096, activation=ReLU, kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)),
    Dropout(0.6),
    Dense(4096, activation=ReLU, kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)),
    Dropout(0.6),

    # tf.keras.layers.Dense(8192, activation='relu', kernel_regularizer=l2(0.001)),
    # tf.keras.layers.Dropout(0.6),
    # tf.keras.layers.Dense(8192, activation='relu', kernel_regularizer=l2(0.001)),
    # tf.keras.layers.Dropout(0.6),
    # tf.keras.layers.Dense(8192, activation='relu', kernel_regularizer=l2(0.001)),
    # tf.keras.layers.Dropout(0.6),
    # tf.keras.layers.Dense(8192, activation='relu', kernel_regularizer=l2(0.001)),
    # tf.keras.layers.Dropout(0.6),
    Lambda(lambda  x: K.l2_normalize(x,axis=1))
])

class DistanceLayer(layers.Layer):
    """
    This layer is responsible for computing the distance between the anchor
    embedding and the positive embedding, and the anchor embedding and the
    negative embedding.
    """

    def __init__(self, **kwargs):
        super().__init__(**kwargs)

    def call(self, anchor, positive, negative):
        ap_distance = tf.reduce_sum(tf.square(anchor - positive), -1)
        an_distance = tf.reduce_sum(tf.square(anchor - negative), -1)
        return (ap_distance, an_distance)


anchor_input = layers.Input(name="anchor", shape=(1280))
positive_input = layers.Input(name="positive", shape=(1280))
negative_input = layers.Input(name="negative", shape=(1280))

distances = DistanceLayer()(
    embedding(anchor_input),
    embedding(positive_input),
    embedding(negative_input),
)

model = Model(
    inputs=[anchor_input, positive_input, negative_input], outputs=distances
)

# model.summary()

In [35]:
imgs = 2
# pred = model.predict((np.random.rand(imgs,1280),np.random.rand(imgs,1280),np.random.rand(imgs,1280)))
# print(pred)

In [36]:
features = {}
# load the features dictionary from the file
with open('features_xl.pickle', 'rb') as handle:
    features = pickle.load(handle)

In [37]:
print(features["02461"])
print(features["02461"].shape)

[ 0.14465111  0.09083702  0.41782254 ...  2.7458491   1.6808972
 -0.00417358]
(1280,)


In [38]:
#convert the features dictionary to a tensorflow hash table
keys = tf.constant([key for key in features.keys()])
values = tf.constant([val for val in features.values()])

table = tf.lookup.experimental.DenseHashTable(
    value_dtype=tf.float32,
    key_dtype=tf.string,
    empty_key="empty_key",
    deleted_key="deleted_key",
    default_value=[-1]*1280,
    )

table.insert(keys, values)

In [39]:
#load strings from train_triplets.txt
train_triplets = np.loadtxt('./train_triplets.txt', dtype=str,delimiter = ' ')


dataset = tf.data.Dataset.from_tensor_slices(train_triplets)


@tf.function
def load_image(inputs):
    filenames = inputs
    # print(filenames)


    # print(table[filenames[0]])
    anchor = table[filenames[0]]
    positive = table[filenames[1]]
    negative = table[filenames[2]]

    output = (anchor,positive,negative)
    
    return output

dataset = dataset.map(load_image).cache().shuffle(buffer_size=1000).batch(batch_size).prefetch(20)

In [40]:
# a,p,n = np.loadtxt('./train_triplets.txt', dtype=str,delimiter = ' ',unpack=True)
# print(a.shape)

# train_triplets = ([features[s] for s in a], [features[s] for s in p], [features[s] for s in n])
# print(len(train_triplets))
# print(len(train_triplets[0]))
# print(len(train_triplets[0][0]))

In [41]:
# dataset = tf.data.Dataset.from_tensor_slices(train_triplets)

# dataset = dataset.cache().batch(batch_size).prefetch(300*batch_size)#.shuffle(300*batch_size)


In [42]:
# next(iter(dataset))

In [43]:
# # This function will plot images in the form of a grid with 1 row and 5 columns where images are placed in each column.
# def plotImages(images_arr):
#     fig, axes = plt.subplots(1, 5, figsize=(20,20))
#     axes = axes.flatten()
#     for img, ax in zip(images_arr, axes):
#         ax.imshow(img)
#     plt.tight_layout()
#     plt.show()

# augmented_images = [my_gen[0][0][0] for i in range(5)]
# plotImages(augmented_images)

In [44]:
class SiameseModel(Model):
    """The Siamese Network model with a custom training and testing loops.

    Computes the triplet loss using the three embeddings produced by the
    Siamese Network.

    The triplet loss is defined as:
       L(A, P, N) = max(‖f(A) - f(P)‖² - ‖f(A) - f(N)‖² + margin, 0)
    """

    def __init__(self, siamese_network, margin=0.5):
        super(SiameseModel, self).__init__()
        self.siamese_network = siamese_network
        self.margin = margin
        self.loss_tracker = metrics.Mean(name="loss")

    def call(self, inputs):
        return self.siamese_network(inputs)

    def train_step(self, data):
        # GradientTape is a context manager that records every operation that
        # you do inside. We are using it here to compute the loss so we can get
        # the gradients and apply them using the optimizer specified in
        # `compile()`.
        with tf.GradientTape() as tape:
            loss = self._compute_loss(data)

        # Storing the gradients of the loss function with respect to the
        # weights/parameters.
        gradients = tape.gradient(loss, self.siamese_network.trainable_weights)

        # Applying the gradients on the model using the specified optimizer
        self.optimizer.apply_gradients(
            zip(gradients, self.siamese_network.trainable_weights)
        )

        # Let's update and return the training loss metric.
        self.loss_tracker.update_state(loss)
        return {"loss": self.loss_tracker.result()}

    def test_step(self, data):
        loss = self._compute_loss(data)

        # Let's update and return the loss metric.
        self.loss_tracker.update_state(loss)
        return {"loss": self.loss_tracker.result()}

    def _compute_loss(self, data):
        # The output of the network is a tuple containing the distances
        # between the anchor and the positive example, and the anchor and
        # the negative example.
        ap_distance, an_distance = self.siamese_network(data)

        # Computing the Triplet Loss by subtracting both distances and
        # making sure we don't get a negative value.
        loss = ap_distance - an_distance
        loss = tf.maximum(loss + self.margin, 0.0)
        return loss

    @property
    def metrics(self):
        # We need to list our metrics here so the `reset_states()` can be
        # called automatically.
        return [self.loss_tracker]

In [45]:
siamese_model = SiameseModel(model,margin=0.5)
# siamese_model.compile(optimizer=tf.keras.optimizers.Adam(0.0001))
# siamese_model.compile(optimizer=tf.keras.optimizers.Adam(0.0000005))
siamese_model.compile(optimizer=tf.keras.optimizers.Adam(0.000001))


# class MyExponentialDecay(tf.keras.optimizers.schedules.ExponentialDecay):
#   def __call__(self, step):
#     return 1e-2 * super().__call__(step)

# initial_learning_rate = 1e-2
# wd = MyExponentialDecay(
#     initial_learning_rate,
#     decay_steps=14,
#     decay_rate=0.8,
#     staircase=True)


# step = tf.Variable(0, trainable=False)
# schedule = tf.optimizers.schedules.PiecewiseConstantDecay([10000, 15000], [1e-0, 1e-1, 1e-2])
# lr = 1e-1 * schedule(step)
# # wd = lambda: 1e-4 * schedule(step)
# opt = tfa.optimizers.AdamW(learning_rate=lr, weight_decay=wd)

# siamese_model.compile(optimizer=opt)


In [46]:

#this is needed, otherwise nothing works
pred = siamese_model.predict(next(iter(dataset)))
print(pred)

(array([1.0158019 , 0.9109661 , 1.2380718 , 0.8157411 , 1.1505013 ,
       1.1772248 , 1.097458  , 0.70524263, 1.029724  , 0.67237407,
       1.0944903 , 1.1234801 , 1.1208537 , 0.76909935, 0.68197465,
       1.1969228 , 0.9064738 , 0.78684723, 1.2427619 , 0.7527365 ,
       0.9580539 , 1.2804571 , 1.1950073 , 0.8814553 ], dtype=float32), array([1.0534555 , 1.2697401 , 1.1218408 , 1.0091112 , 1.0669051 ,
       1.1927825 , 1.001681  , 0.8769452 , 1.1582739 , 0.9604594 ,
       0.92310107, 1.2607124 , 0.82224506, 1.2321987 , 1.050529  ,
       0.71917474, 1.1692468 , 1.0420399 , 1.1962942 , 0.91715527,
       1.1158857 , 1.2549804 , 1.1705428 , 1.0881376 ], dtype=float32))


In [47]:
# print(my_gen.__getitem__(5)[0])

In [48]:
train_steps_per_epoch = int((59520)/batch_size)
# train_steps_per_epoch = int((59520*5)/batch_size)

# checkpoint_filepath = './checkpoint'
checkpoint_filepath = './checkpoints/checkpoint-{epoch}/'
# checkpoint_filepath = "saved-model-{epoch}.hdf5"
# checkpoint_filepath = "saved-model-{epoch}"
#save checkpoint after every epoch
model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    # save_weights_only=True,
)


siamese_model.fit(dataset, epochs=1000,steps_per_epoch=train_steps_per_epoch,batch_size=batch_size,callbacks=[model_checkpoint_callback])
# siamese_model.fit(train_triplets, epochs=2,steps_per_epoch=train_steps_per_epoch,batch_size=batch_size,callbacks=[model_checkpoint_callback])


Epoch 1/1000
2480/2480 [==============================] - 42s 17ms/step - loss: 0.3537
Epoch 2/1000
2480/2480 [==============================] - 44s 18ms/step - loss: 0.2967
Epoch 3/1000
2480/2480 [==============================] - 45s 18ms/step - loss: 0.2848
Epoch 4/1000
2480/2480 [==============================] - 41s 17ms/step - loss: 0.2775
Epoch 5/1000
2480/2480 [==============================] - 39s 16ms/step - loss: 0.2715
Epoch 6/1000
2480/2480 [==============================] - 37s 15ms/step - loss: 0.2664
Epoch 7/1000
2480/2480 [==============================] - 35s 14ms/step - loss: 0.2617
Epoch 8/1000
2480/2480 [==============================] - 33s 13ms/step - loss: 0.2573
Epoch 9/1000
2480/2480 [==============================] - 34s 14ms/step - loss: 0.2533
Epoch 10/1000
2480/2480 [==============================] - 36s 14ms/step - loss: 0.2495
Epoch 11/1000
2480/2480 [==============================] - 33s 13ms/step - loss: 0.2459
Epoch 12/1000
2480/2480 [================

Error: Canceled future for execute_request message before replies were done

In [ ]:

# model_path = "siamese_model"
# tf.saved_model.save(siamese_model, model_path)
